# Integration with External Verilog
This guide is targeted towards users working with existing Verilog designs (as
opposed to Python/Magma circuits).

The approach relies on Magma's external Verilog integration features to
construct a Magma circuit representation that can then be used with a
`fault.Tester` object. This approach enables the use of most of fault's
features, except for logic that relies on descending into a design hierarchy
(e.g. peeking and poking sub-instance ports).

The first, simplest approach is to use `m.define_from_verilog` (which
takes a string containing verilog code) or
`m.define_from_verilog_file` (which takes a path to a file that contains verilog code)
to import a verilog design into Magma.  

Here's an example using `m.define_from_verilog`

In [1]:
import magma as m
import logging
logging.basicConfig(level=logging.INFO)
import fault

# NOTE: define_from_verilog returns a list of modules 
# (since there could be multiple), so in this case we 
# simply index the first and only module with `[0]`
foo = m.define_from_verilog("""\
module foo(input I, output O);
assign O = I;
endmodule
""", target_modules=["foo"])[0]

print(f"Imported as magma circuit: {foo}")

tester = fault.Tester(foo)
tester.circuit.I = 1
tester.eval()
tester.circuit.O.expect(1)
tester.compile_and_run("verilator")

Imported as magma circuit: foo(I: In(Bit), O: Out(Bit))


INFO:root:Running tester...
INFO:root:Success!


Here's an example using `m.define_from_verilog_file`

In [2]:
# write verilog string to a file
with open("foo.v", "w") as f:
    f.write("""\
module foo(input I, output O);
assign O = I;
endmodule
""")
    
foo = m.define_from_verilog_file("foo.v", target_modules=["foo"])[0]
print(f"Imported as magma circuit: {foo}")

tester = fault.Tester(foo)
tester.circuit.I = 1
tester.eval()
tester.circuit.O.expect(1)
tester.compile_and_run("verilator")

Imported as magma circuit: foo(I: In(Bit), O: Out(Bit))


INFO:root:Running tester...
INFO:root:Success!


An alternative to using `define_from_verilog` is to use `declare_from_verilog` to import a module interface, and provide the implementation to the simulator by copying the source verilog file into the simulation directory.  This is useful when the source file contains code that is not supported by Magma's Verilog parser (e.g. advanced system verilog features), or when parsing takes a long time (e.g. a post-synthesis netlist file).

In [3]:
with open("foo_stub.v", "w") as f:
    f.write("""\
module foo(input I, output O);
endmodule
""")

# You can similarly use declare_from_verilog with a Verilog string
foo = m.declare_from_verilog_file("foo_stub.v", target_modules=["foo"])[0]
print(f"Imported as magma circuit: {foo}")


tester = fault.Tester(foo)
tester.circuit.I = 1
tester.eval()
tester.circuit.O.expect(1)

import tempfile
import shutil

with tempfile.TemporaryDirectory() as dir_:
    # Copy actual implementation to test directory
    shutil.copy("foo.v", dir_)
    # Set test directory with directory= kwarg
    # Note: we also tell magma to skip compilation (skip_compile=True)
    # since the verilog file is already present in the test directory 
    # (copied in the previous line)
    tester.compile_and_run("verilator", directory=dir_, skip_compile=True)

INFO:root:Running tester...


Imported as magma circuit: foo(I: In(Bit), O: Out(Bit))


INFO:root:Success!


A similar approach is to declare the interface using magma. This has the advantage of providing the capability of writing a sophisticated interface generator for your external module (e.g. if you're integrating with an external generator framework).  In doing this, you may find it useful to first define/declare a Circuit for the verilog module (as done above) and use the basic corresponding types.  Then, in a magma circuiat wrapper, you can use more complex types and wire them up to the underlying verilog ports.

In [4]:
# Declare magma circuit with the same name and equivalent interface
# to a verilog circuit
class foo(m.Circuit):
    io = m.IO(I=m.In(m.Bit), O=m.Out(m.Bit))

tester = fault.Tester(foo)
tester.circuit.I = 1
tester.eval()
tester.circuit.O.expect(1)

import tempfile
import shutil

with tempfile.TemporaryDirectory() as dir_:
    # Copy actual implementation to test directory
    shutil.copy("foo.v", dir_)
    # Set test directory with directory= kwarg
    tester.compile_and_run("verilator", directory=dir_, skip_compile=True)

INFO:root:Running tester...
INFO:root:Success!
